# Iterative SQD (iSQD)

In [3]:
from copy import deepcopy
from functools import partial
import h5py
import numpy as np

import pyscf
from pyscf import ao2mo
from pyscf.tools.fcidump import from_scf, read

import openfermion as of

from qiskit import QuantumRegister, QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.qasm2 import dumps
import qiskit_ibm_runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_addon_sqd.fermion import diagonalize_fermionic_hamiltonian, solve_sci_batch, SCIResult

import ffsim

from adaptvqe.utils import hamiltonian_from_fcidump
from adaptvqe.hamiltonians import FermionicHamiltonian
from adaptvqe.pools import DVG_CEO
from adaptvqe.algorithms.adapt_vqe import TensorNetAdapt


In [4]:
ibm_computer: str = "ibm_fez"

service = qiskit_ibm_runtime.QiskitRuntimeService(channel="local")
computer = service.backend()
sampler = Sampler(computer)

/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/qiskit_ibm_runtime/fake_provider/backends/nighthawk/fake_nighthawk.py:76: UserWarning: Properties of fake_nighthawk are not intended to represent typical nighthawk error values.
  warnings.warn(


In [5]:
# mol = pyscf.gto.Mole()
# geom = of.chem.geometry_from_pubchem("LiH")
# basis = "sto-3g"
# symmetry = "C1"
# multiplicity = 1
# charge = 0
# mol.build(
#     atom=geom,
#     basis=basis,
#     symmetry=symmetry,
# )
# mf = pyscf.scf.RHF(mol)
# mf.kernel()
# lih_fname = "data/lih.fcidump"
# from_scf(mf, lih_fname)

In [6]:
lih_fname = "atp_0_fcidump/atp_0_be1_f0"

In [7]:
h_fcidump, norb, nelec = hamiltonian_from_fcidump(lih_fname)
h = FermionicHamiltonian(h_fcidump, "atp", nelec)
nq = h.n
print(f"Hamiltonian has {nq} qubits.")

Hamiltonian has 24 qubits.


In [8]:
# Since we have alpha and beta e-'s, an n-qubit Hamiltonian as n/2 spatial orbitals.
assert h.n % 2 == 0
num_orbitals = h.n // 2

In [ ]:
# TODO Get the exact energy from PySCF.
exact_energy = h.ground_energy
print(f"Got exact energy {exact_energy}.")

In [9]:
# Get one- and two-body integrals.
fci_read = read(lih_fname)
h1 = fci_read["H1"]
h2_packed = fci_read["H2"]
h2 = ao2mo.restore(1, h2_packed, num_orbitals)  # (norb,norb,norb,norb)
n_electrons = fci_read["NELEC"]
ecore = fci_read["ECORE"]
spin = 0 # TODO How would I know from the FCIDUMP alone?
num_elec_a = (n_electrons + spin) // 2
num_elec_b = (n_electrons - spin) // 2
nelec = (num_elec_a, num_elec_b)

Parsing atp_0_fcidump/atp_0_be1_f0


In [8]:
import time

In [9]:
start = time.monotonic()
pool = DVG_CEO(n=h.n)
stop = time.monotonic()
print(f"Finished building pool in {stop - start} seconds. Has {len(pool.operators)} operators.")

DVG_CEO init
CEO init
Track parents
QE init
OperatorPool init
Calling self.create_operators
QE create_operators
QE create_singles, self.n = 24
p, q = 0 2
p, q = 0 4
p, q = 0 6
p, q = 0 8
p, q = 0 10
p, q = 0 12
p, q = 0 14
p, q = 0 16
p, q = 0 18
p, q = 0 20
p, q = 0 22
p, q = 1 3
p, q = 1 5
p, q = 1 7
p, q = 1 9
p, q = 1 11
p, q = 1 13
p, q = 1 15
p, q = 1 17
p, q = 1 19
p, q = 1 21
p, q = 1 23
p, q = 2 4
p, q = 2 6
p, q = 2 8
p, q = 2 10
p, q = 2 12
p, q = 2 14
p, q = 2 16
p, q = 2 18
p, q = 2 20
p, q = 2 22
p, q = 3 5
p, q = 3 7
p, q = 3 9
p, q = 3 11
p, q = 3 13
p, q = 3 15
p, q = 3 17
p, q = 3 19
p, q = 3 21
p, q = 3 23
p, q = 4 6
p, q = 4 8
p, q = 4 10
p, q = 4 12
p, q = 4 14
p, q = 4 16
p, q = 4 18
p, q = 4 20
p, q = 4 22
p, q = 5 7
p, q = 5 9
p, q = 5 11
p, q = 5 13
p, q = 5 15
p, q = 5 17
p, q = 5 19
p, q = 5 21
p, q = 5 23
p, q = 6 8
p, q = 6 10
p, q = 6 12
p, q = 6 14
p, q = 6 16
p, q = 6 18
p, q = 6 20
p, q = 6 22
p, q = 7 9
p, q = 7 11
p, q = 7 13
p, q = 7 15
p, q = 7 17
p

In [13]:
(stop - start) / 60 / 60

4.432998526899415

In [22]:
max_mpo_bond = 8
adapt_mps_bond = 16
my_adapt = TensorNetAdapt(
    pool=pool,
    custom_hamiltonian=h,
    max_adapt_iter=1,
    recycle_hessian=True,
    tetris=True,
    verbose=True,
    threshold=0.1,
    max_mpo_bond=max_mpo_bond,
    max_mps_bond=adapt_mps_bond,
)
my_adapt.initialize()

: 

In [ ]:
circuits = []
adapt_energies = []
for i in range(3):
    print(f"On iteration {i}.")
    my_adapt.run_iteration()
    data = my_adapt.data
    ansatz_circuit = pool.get_circuit(my_adapt.indices, my_adapt.coefficients)
    print("coefficients:", my_adapt.coefficients)
    print("indices:", my_adapt.indices)
    # Prepare the HF reference state, then add the Ansatz circuit.
    q = QuantumRegister(2 * num_orbitals)
    circuit = QuantumCircuit(q)
    circuit.append(ffsim.qiskit.PrepareHartreeFockJW(num_orbitals, nelec), q)
    circuit = circuit.compose(ansatz_circuit)
    circuit.measure_all()
    circuits.append(circuit)
    adapt_energies.append(my_adapt.energy)

In [ ]:
circuits[-1].draw(fold=-1)

In [ ]:
spin_a_layout = list(range(0, 12))
spin_b_layout = [12, 13, 14, 15, 19, 35, 34, 33, 32, 31, 30, 29]
initial_layout = spin_a_layout + spin_b_layout

In [ ]:
pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=computer, initial_layout=initial_layout[:nq]
)
pass_manager.pre_init = ffsim.qiskit.PRE_INIT

to_run = []
for circuit in circuits:
    # TODO: Remove barriers.
    compiled = pass_manager.run(circuit)
    print(compiled.count_ops())
    to_run.append(compiled)

In [ ]:
nshots: int = 10_000

In [ ]:
job = sampler.run(to_run, shots=nshots)  # TODO: Set nshots for each circuit.

In [ ]:
bit_arrays = [result.data.meas for result in job.result()]
counts_list = [bit_array.get_counts() for bit_array in bit_arrays]

In [ ]:
energies = []
errors = []
for bit_array in bit_arrays:
    # SQD options
    energy_tol = 1e-5
    occupancies_tol = 1e-6
    max_iterations = 20
    rng = np.random.default_rng(1)

    # Eigenstate solver options
    num_batches = 2
    samples_per_batch = 1000
    symmetrize_spin = True
    carryover_threshold = 1e-4
    max_cycle = 200

    # Pass options to the built-in eigensolver. If you just want to use the defaults,
    # you can omit this step, in which case you would not specify the sci_solver argument
    # in the call to diagonalize_fermionic_hamiltonian below.
    sci_solver = partial(solve_sci_batch, spin_sq=0.0, max_cycle=max_cycle)

    # List to capture intermediate results
    result_history = []


    def callback(results: list[SCIResult]):
        result_history.append(results)
        iteration = len(result_history)
        print(f"Iteration {iteration}")
        for i, result in enumerate(results):
            print(f"\tSubsample {i}")
            print(f"\t\tEnergy: {result.energy + ecore}") # + nuclear_repulsion_energy}")
            print(f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")


    result = diagonalize_fermionic_hamiltonian(
        h1,
        h2,
        bit_array,
        samples_per_batch=samples_per_batch,
        norb=num_orbitals,
        nelec=nelec,
        num_batches=num_batches,
        energy_tol=energy_tol,
        occupancies_tol=occupancies_tol,
        max_iterations=max_iterations,
        sci_solver=sci_solver,
        symmetrize_spin=symmetrize_spin,
        carryover_threshold=carryover_threshold,
        callback=callback,
        seed=rng,
    )

    min_e = [
        min(result, key=lambda res: res.energy).energy + ecore # + nuclear_repulsion_energy
        for result in result_history
    ]
    approx_energy = min(min_e)
    err = abs(min(min_e) - exact_energy)
    print(approx_energy, err)
    energies.append(approx_energy)
    errors.append(err)

In [ ]:
qasm_strings = []
for circuit in circuits:
    isa_circuit = pass_manager.run(circuit)
    qasm_str = dumps(isa_circuit)
    qasm_strings.append(qasm_str)



In [ ]:
molec_name = "LiH"
f = h5py.File(f"data/{molec_name}.hdf5", "w")
f.create_dataset("energies", data=np.array(energies))
f.create_dataset("circuit_qasm", data=qasm_strings)